### Press shift+enter to run the codes in the cell

import useful libraries

In [1]:
#import tensorflow as tf
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
#from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import scipy.ndimage
#%matplotlib inline

load data and transform them into workable numpy array

In [2]:
pathX = "MNIST_Xtrain.csv"
pathy = "MNIST_ytrain.csv"
pathXreal = 'MNIST_Xtest.csv'
#df_X = pd.read_csv(pathX, header = None).transpose()
df_y = pd.read_csv(pathy, header = None)
df_X = pd.read_csv(pathX, header = None)
df_Xreal = pd.read_csv(pathXreal, header = None)

X = df_X.values
y = df_y.values.flatten() # flatten y to be row vector
# I call the test values as real values since I need to use 'test' in train-test split, X_test=Xreal
Xreal = df_Xreal.values 

## Stage 1. Data augmentation:

This is the data preprocessing stage, where I used only data augmentation, since normalization is already done, and feature selection gets rid off the pixel-grids. (The result with feature selection turns out to be 94% roughly, which is not bad but it is not better than non-feature selection stage which is this one I have currently.) So I decide to not to have feature selection stage.

The Augmentation class consists of 4 methods for augmenting the original dataset: (1) rotation of the original dataset by 10 degrees clock-wise, (2) 10 degrees counter-clock wise, (3) translation to the right by 1 pixel, (4) and a combination of rotate10cw and translation. This will result in an augmented version of 300,000 data from the original 60,000 training dataset. The purpose is to train the learner to have it see different characteristics or styles of the digits so that it has better performance on the unseen dataset.

In [3]:
class Augmentation:
    
    def __init__(self, X):
        self.X = X
        self.R10cw = np.concatenate((self.X, self.rotate10cw(self.X)), axis=0)
        self.R10cwT = np.concatenate((self.R10cw, self.translation(self.X)), axis=0)
        self.R10cwTR10ccw = np.concatenate((self.R10cwT, self.rotate10ccw(self.X)), axis=0)
        self.R10cwTR10ccwANDRT = np.concatenate((self.R10cwTR10ccw, 
                                                 self.rotate10cw(self.translation(self.X))), axis=0)
    
    # rotate image 10 degrees clock-wise
    def rotate10cw(self, X):
        # create empty array for storing the rotation by 10-cw degrees
        X_rot10cw = np.empty([self.X.shape[0], self.X.shape[1]]) 
        
        for i,x in enumerate(self.X):
            x_rot10cw = scipy.ndimage.interpolation.rotate(np.transpose(np.reshape(x,(28,28))), 
                                                         -10, axes=(1, 0), reshape=False)
            x_rot10cw = np.transpose(x_rot10cw) #make column go first
            X_rot10cw[i] = np.reshape(x_rot10cw.flatten(),(1,-1))
        
        return X_rot10cw 
    
    # rotate the image 10 degrees ccw (counter-clock wise)
    def rotate10ccw(self, X):
        # create empty array for storing rotation by 10 degrees-ccw image
        X_rot10ccw = np.empty([self.X.shape[0], self.X.shape[1]])
        
        for i,x in enumerate(self.X):
            x_rot10ccw = scipy.ndimage.interpolation.rotate(np.transpose(np.reshape(x,(28,28))), 
                                                         10, axes=(1, 0), reshape=False)
            x_rot10ccw = np.transpose(x_rot10ccw)
            X_rot10ccw[i] = np.reshape(x_rot10ccw.flatten(),(1,-1))
        
        return X_rot10ccw
        
    # translate the image to the right by 1 pixel 
    def translation(self, X):
        # create an empty array for storing the shift-right-by-1-pixel image
        X_shift = np.empty([self.X.shape[0], self.X.shape[1]])
        
        for i,x in enumerate(self.X):
            x_shift=scipy.ndimage.interpolation.shift(np.transpose(np.reshape(x,(28,28))), 1.)
            x_shift = np.transpose(x_shift)
            X_shift[i] = np.reshape(x_shift.flatten(),(1,-1))
        
        return X_shift

In [4]:
# create an object of the Augmentation class and call it X_transform
X_transform = Augmentation(X)

#print(X_transform.R10cwTR10ccwANDRT.shape)

# increase the dimension of y as well
y_R = np.concatenate((y, y), axis=0)
y_RT = np.concatenate((y_R,y), axis=0)
y_RTR = np.concatenate((y_RT,y), axis=0)
y_RTRrt = np.concatenate((y_RTR, y), axis=0)

#print(y_RTRrt.shape)

## Stage 2. Model Selection:

##### ***Please bare with this CrossValidate class, it will take approximately 4.5 days...*** Thank you.

1). The classifier used is Support Vector Machine which maximizes the Lagrangian dual objective function:

\begin{equation*}
L_D = \sum_{i=1}^N \alpha_i - \frac{1}{2} \sum_{i=1}^N \sum_{i'=1}^N \alpha_i \alpha_{i'} y_i y_{i'} \langle h(x_i),h(x_{i'}) \rangle
\end{equation*}

subjected to $C \ge \alpha_i \ge 0$ and $\sum_{i=1}^N \alpha_i y_i =0$, and I applied some transformation $h(x)$ to the input vector space, in which I used the Radial Basis as the kernel functions:

\begin{equation*}
K(x,x') = \langle h(x), h(x') \rangle = e^{(-\gamma \|x-x' \|^2)}
\end{equation*}

between two sample vectors $x$ and $x'$. The Kernal function does not depend on the actual transformation of $h(x)$, it only depends on the inner product between $h(x)$ and $h(x')$ for any $x$, $x'$ $\in$ $X_{train}$. (I also tried ploy kernel with degree 2 and linear kernel and I tested all of them, rbf gives the highest score)

2). 5-fold cross validation accompanied with a parameter grid-search algorithm, where the hyperparamters need to be determined are the penalization $C$ and the rbf parameter $\gamma$. A grid-search is run through all the possible combinations for $C \in$ [10,11,12,13] and $\gamma \in$ [0.01, 0.015] on a 5-fold cross validation scheme and the best $C-\gamma$ pair, i.e. the ones that gives the highest CV accuracy is returned. And this $C-\gamma$ pair is used on the entire X  dataset to obtain a final classifier to predict the labels for Xreal, the test dataset. I intially tried a coarse grid-search with $C$ = [10, 20, 30], and $C$=10 gave the highest accuracy and then now, I am doing a finer grid-search with $C$=[10,11,12,13] with $\gamma$ unchanged. I also tried with other $\gamma$ values as well, but it turns out that any value in between 0.01 and 0.03 works better among others, so I just picked two values 0.01 and 0.015 for the grid-search algorithm so that it takes relatively less time to run, since the algorithm needs to run 8 $C-\gamma$ pairs and 5 times for each of them. (note: if you want to see some scratch work I have done, please check out the other_parallels.folder in which I attached some representive scratches among all the other values I tried)

In [8]:
class CrossValidate:
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
        C_options = [10, 11, 12, 13]
        gamma_options = [0.01, 0.015]
        print('selecting the parameters using 5-fold CV...')
        print('please wait..')
        self.candidate, cv_score = self.model_selection(self.X, self.y, C_options, gamma_options, 5)
        
        print('C-gamma pair on coarse grid-search: ' + str(self.candidate))
        print('average accuracy of the CV: ' + str(cv_score))
    
    # paramter tuning
    def model_selection(self, X, y, C_options, gamma_options, nfolds):
        parameters = {'C': C_options, 'gamma' : gamma_options}
        grid_search = GridSearchCV(SVC(kernel='rbf', random_state=0), parameters, cv=nfolds)
        grid_search.fit(X, y)
        candidate = grid_search.best_params_
        cv_score = grid_search.best_score_
        
        return candidate, cv_score    

Feed the 300,000 training dataset to my CrossValidate class and return the $C-\gamma$ candidate

In [3]:
result = CrossValidate(X_transform.R10cwTR10ccwANDRT, y_RTRrt)

result.candidate

Once have the best $C$ and $\gamma$ values, train on the entire X to get my final classifier and make predictions on Xreal

In [7]:
svc = SVC(C=result.candidate.get('C'), 
          gamma=result.candidate.get('gamma'), 
          kernel='rbf', random_state=0)

svc.fit(X_transform.R10cwTR10ccwANDRT, y_RTRrt)

# make predictions on X_test as provided in A2
predictions = svc.predict(Xreal)

Create a panada dataframe and save as .csv file. Done

In [ ]:
testPred = pd.DataFrame({'Digit':predictions, 'ImageID':np.arange(1,Xreal.shape[0]+1)})
#print(testPred)

submission = pd.DataFrame({'ImageID': testPred['ImageID'], 'Digit':testPred['Digit']})
filename = 'prediction.csv'
submission.to_csv(filename,index=False)

print('Saved file: ' + filename)